### 1. Import Table with products and links

In [2]:
import pandas as pd

products = pd.read_excel('Produtos.xlsx')
display(products)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,NaN,NaN
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,NaN,NaN


### 2. Get Cellphone Prices from Links

In [3]:
def price_to_format(price_string):
    price_string = price_string.replace('.','').replace('R$','').replace(',','.')
    return float(price_string)

In [4]:
from selenium import webdriver
import time

#getting weblinks
driver = webdriver.Chrome()
url_amazon_cell = products['Amazon'].values[0]
url_americanas_cell = products['Lojas Americanas'].values[0]
url_magalu_cell = products['Magazine Luiza'].values[0]

#Amazon price
driver.get(url_amazon_cell)
price_cell_ama = driver.find_element_by_id('priceblock_ourprice').text
price_cell_ama = price_to_format(price_cell_ama)


In [5]:
#Americanas price
#driver.get(url_americanas_cell)
#time.sleep(5)
#price_cell_amer = driver.find_element_by_class_name('src__Title-sc-1u9ae8m-1 fETrEF').text
price_cell_amer = 4104.21
#print(price_cell_amer)
#print(type(price_cell_amer))


In [6]:
#Magalu prices
driver.get(url_magalu_cell)
price_cell_ma = driver.find_element_by_class_name('price-template__text').text
price_cell_ma = price_to_format(price_cell_ma)

### 3. Get TV Prices from Links

In [7]:
driver_tv = webdriver.Chrome()
url_amazon_tv = 'https://www.amazon.com.br/LG-50UN731C-Assistente-Virtual-Bluetooth/dp/B08FJCW8H3/ref=pd_lpo_504_t_0/133-7858174-5194815?_encoding=UTF8&pd_rd_i=B08FJCW8H3&pd_rd_r=330af3ee-e4bc-4b4a-8821-f6426baea40c&pd_rd_w=rX31W&pd_rd_wg=CnS09&pf_rd_p=cfa1789c-e63b-406f-a502-2b3819ce5a27&pf_rd_r=DN3AVGJ2TRR887SCBFE8&psc=1&refRID=DN3AVGJ2TRR887SCBFE8'
url_americanas_tv = 'https://www.americanas.com.br/produto/495161500?pfm_carac=smart-tv-led-pro-50-ultra-hd-4k-lg-50um751c0sb-thinq-ai-4-hdmi-2-usb-wi-fi&pfm_index=3&pfm_page=search&pfm_pos=grid&pfm_type=search_page'
url_magalu_tv = products['Magazine Luiza'].values[1]

#Amazon Price
driver_tv.get(url_amazon_tv)
price_tv_ama = driver_tv.find_element_by_id('priceblock_ourprice').text
price_tv_ama = price_to_format(price_tv_ama)

In [8]:
#Americanas Price
#driver_tv.get(url_americanas_tv)
#price_tv_amer = driver_tv.find_element_by_class_name('src__BestPrice-sc-1jvw02c-5 cBWOIB priceSales').text
price_tv_amer = 2599.00

In [9]:
#Magazine Luiza price
driver_tv.get(url_magalu_tv)
price_tv_ma = driver_tv.find_element_by_class_name('price-template__text').text
price_tv_ma = price_to_format(price_tv_ma)

### 4. Input prices in DataFrame and Store (link)

In [11]:
list_cell = [(price_cell_ama,'Amazon'),(price_cell_amer,'Lojas Americanas'),(price_cell_ma,'Magazine Luiza')]
list_tv = [(price_tv_ama,'Amazon'),(price_tv_amer,'Lojas Americanas'),(price_tv_ma,'Magazine Luiza')]
list_cell.sort()
list_tv.sort()
products.loc[0,'Preço Atual'] = list_cell[0][0]
products.loc[0,'Local'] = list_cell[0][1]
products.loc[1,'Preço Atual'] = list_tv[0][0]
products.loc[1,'Local'] = list_tv[0][1]
display(products)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,3905.07,Magazine Luiza
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,2599.00,Lojas Americanas


### 5.  Save Spreadsheet and Send Email in case of 20% Discount

In [16]:
import yagmail

products.to_excel(r'Produtos.xlsx')
price_current_cell = products.loc[0,'Preço Atual']
price_original_cell = products.loc[0,'Preço Original']
price_current_tv = products.loc[1,'Preço Atual']
price_original_tv = products.loc[1,'Preço Original']

if price_current_cell <= 0.8 * price_original_cell or price_current_tv <= 0.8 * price_original_tv :
    user = yagmail.SMTP(user = 'youremail@gmail.com',password = '1234')
    if price_current_cell <= 0.8 * price_original_cell:
        text = f'''
        Bom dia prezado, como está?
        
        Encaminho esse email para alertar que existe uma boa oportunidade de compra de um Iphone 11. O preço atual é de R${price_current_cell} e pode ser encontrado na loja {products.loc[0,'Local']}, 
        
        Qualque dúvida, fico à disposição.
        
        Att. Felipe
        '''
        user.send(to = 'youremail+1@gmail.com', subject = 'Oportunidade de Compra IPHONE11',contents = text)
    if price_current_tv <= 0.8 * price_original_tv:
        text = f'''
        Bom dia prezado, como está?
        
        Encaminho esse email para alertar que existe uma boa oportunidade de compra de uma TV LED 50' LG. O preço atual é de R${price_current_tv} e pode ser encontrado na loja {products.loc[1,'Local']}, 
        
        Qualque dúvida, fico à disposição.
        
        Att. Felipe
        '''
        user.send(to = 'youremail+1@gmail.com', subject = 'Oportunidade de Compra TVLED',contents = text)
    